In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *

In [0]:
#dbutils.secrets.listScopes()

In [0]:
#dbutils.secrets.list(scope="databricksscope")

In [0]:
spark.conf.set("fs.azure.account.key.optumsadllssa.dfs.core.windows.net",dbutils.secrets.get(scope="databricksscope",key="adlskey"))

In [0]:
#display(dbutils.fs.ls("abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net"))

In [0]:
data = spark.read.csv('abfss://optumrawdataset@optumsadllssa.dfs.core.windows.net/hospital.csv',header=True,inferSchema=True)

In [0]:
#data.show(5,False)

In [0]:
#data.count()

In [0]:
#data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

In [0]:
#data.groupby(['Hospital_id','Hospital_name','city','state','country']).count().where("count>1").show()

In [0]:
data = data.dropDuplicates()

In [0]:
data = data.replace("NaN",None)

In [0]:
#data.show(5,False)

In [0]:
data = data.fillna({'state':"UT"})

In [0]:
#data.show(5,False)

In [0]:
#data.select("city").distinct().show(10)

In [0]:
data = data.replace(['New Delhi'],['Delhi'],'city')

In [0]:
#data.show()

In [0]:

output_container_path = "abfss://optumstagingdata@optumsadllssa.dfs.core.windows.net"
output_blob_folder = "optumstagingdata"
data.coalesce(1).write.format("csv").mode("overwrite").option("header","true").format("com.databricks.spark.csv").save(output_blob_folder)
files = dbutils.fs.ls(output_blob_folder)
output_file = [x for x in files if x.name.startswith("part-")]
dbutils.fs.mv(output_file[0].path,"%s/hospitaldatastaging.csv"%output_container_path)

True